In [12]:
import pandas as pd

# Load the existing CSV file
csv_file = 'target_.csv'  # Replace with your file path if different
df = pd.read_csv(csv_file)

# Add the "Store" column with the value "Whole Foods"
df['Store'] = 'Target'

# Save the updated DataFrame back to the CSV
df.to_csv(csv_file, index=False)

print("Store column added successfully!")


Store column added successfully!


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import re

# Webdriver setup
chrome_path = 'C:\\Users\\nwang\\chromedriver.exe'
service = Service(chrome_path)
driver = webdriver.Chrome(service=service)

# Define categories with search URLs
categories = {
    "Fresh Produce": ["https://www.wholefoodsmarket.com/search?text=fresh+produce",
                     "https://www.wholefoodsmarket.com/search?text=vegetables",
                     "https://www.wholefoodsmarket.com/search?text=fruit"],
    "Meat & Seafood": [
        "https://www.wholefoodsmarket.com/search?text=meat",
        "https://www.wholefoodsmarket.com/search?text=seafood",
        "https://www.wholefoodsmarket.com/search?text=chicken",
        "https://www.wholefoodsmarket.com/search?text=pork"
    ],
    "Snacks": ["https://www.wholefoodsmarket.com/search?text=snacks",
              "https://www.wholefoodsmarket.com/search?text=chips"],
    "Beverages": ["https://www.wholefoodsmarket.com/search?text=beverages",
                 "https://www.wholefoodsmarket.com/search?text=soda"],
    "Breakfast & Cereal": [
        "https://www.wholefoodsmarket.com/search?text=breakfast",
        "https://www.wholefoodsmarket.com/search?text=cereal"
    ],
    "Bakery": ["https://www.wholefoodsmarket.com/search?text=bakery"],
    "Dairy & Eggs": ["https://www.wholefoodsmarket.com/search?text=dairy+and+eggs",
                    "https://www.wholefoodsmarket.com/search?text=milk",
                    "https://www.wholefoodsmarket.com/search?text=cheese"],
    "Frozen": ["https://www.wholefoodsmarket.com/search?text=frozen"]
}

# List to hold scraped product data
data = []

# Regex pattern to find quantities (e.g., "3 lbs", "1.13-2.75 lbs")
quantity_pattern = re.compile(r'(\d+\.?\d*\s*(?:lb|lbs|oz|ct|pk|fl oz))', re.IGNORECASE)

# Loop through each category and its URLs
for category, urls in categories.items():
    for url in urls:  # Handle multiple URLs per category
        print(f"Scraping category: {category}, URL: {url}")
        driver.get(url)
        time.sleep(3)  # Wait for the page to load

        # Scroll to load products
        for _ in range(5):
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(2)

        # Find product elements
        products = driver.find_elements(By.CLASS_NAME, 'w-pie--product-tile__content')

        # Iterate through the products and prices
        for product in products:
            try:
                name = product.find_element(By.XPATH, './/h2[@data-testid="product-tile-name"]').text
            except:
                name = "No name"

            try:
                brand = product.find_element(By.XPATH, './/span[@data-testid="product-tile-brand"]').text
            except:
                brand = "No brand"

            try:
                price_text = product.find_element(By.XPATH, './/span[contains(@class, "text-left bds--heading-5")]').text
            except:
                price_text = "No price"

            # Combine name and brand
            full_name = f"{name} - {brand}" if brand != "No brand" else name

            # Extract quantity from product name
            quantity_match = quantity_pattern.search(name)
            quantity_text = quantity_match.group(1) if quantity_match else "1"  # Default if quantity is not found

            # Append data to the list
            data.append({
                "Category": category,
                "Name": full_name,  # Updated name with brand
                "Price": price_text,
                "Quantity": quantity_text
            })

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('wholefoods.csv', index=False)

# Clean up and close the driver
driver.quit()

print("Data scraping complete. Products saved to 'wholefoods.csv'.")


Scraping category: Fresh Produce, URL: https://www.wholefoodsmarket.com/search?text=fresh+produce
Scraping category: Fresh Produce, URL: https://www.wholefoodsmarket.com/search?text=vegetables
Scraping category: Fresh Produce, URL: https://www.wholefoodsmarket.com/search?text=fruit
Scraping category: Meat & Seafood, URL: https://www.wholefoodsmarket.com/search?text=meat
Scraping category: Meat & Seafood, URL: https://www.wholefoodsmarket.com/search?text=seafood
Scraping category: Meat & Seafood, URL: https://www.wholefoodsmarket.com/search?text=chicken
Scraping category: Meat & Seafood, URL: https://www.wholefoodsmarket.com/search?text=pork
Scraping category: Snacks, URL: https://www.wholefoodsmarket.com/search?text=snacks
Scraping category: Snacks, URL: https://www.wholefoodsmarket.com/search?text=chips
Scraping category: Beverages, URL: https://www.wholefoodsmarket.com/search?text=beverages
Scraping category: Beverages, URL: https://www.wholefoodsmarket.com/search?text=soda
Scraping 

In [2]:
import pandas as pd

# Load the CSV file
csv_file = 'wholefoods.csv'  # Replace with your file path if different
df = pd.read_csv(csv_file)

# Filter out rows where the "Price" column is "No price"
df = df[df['Price'] != "No price"]

# Add the "Store" column with the value "Whole Foods"
df['Store'] = 'Whole Foods'

#drop duplicates
df = df.drop_duplicates(subset='Name', keep='first')


# Save the updated DataFrame back to the CSV
df.to_csv('wholefoods_cleaned.csv', index=False)

print("Rows with 'No price' removed and 'Store' column added. Cleaned data saved to 'wholefoods_cleaned.csv'.")



Rows with 'No price' removed and 'Store' column added. Cleaned data saved to 'wholefoods_cleaned.csv'.


In [20]:
import pandas as pd

# Load the CSV file
csv_file = 'target_.csv'  # Replace with your file path if different
df = pd.read_csv(csv_file)

# Filter out rows where the "Price" column is "max price"
df = df[df['Price'] != "max price"]

# Filter out rows where the price is a range (contains " - ")
df = df[~df['Price'].str.contains(r'\s-\s', na=False)]

# Save the cleaned DataFrame to a new CSV file
df.to_csv('target_cleaned.csv', index=False)

print("Rows with 'max price' and price ranges removed. 'Store' column added. Cleaned data saved to 'target_cleaned.csv'.")


Rows with 'max price' and price ranges removed. 'Store' column added. Cleaned data saved to 'target_cleaned.csv'.


In [3]:
import pandas as pd

# Load the cleaned Whole Foods CSV
wholefoods_file = 'wholefoods_cleaned.csv'
df_wholefoods = pd.read_csv(wholefoods_file)

# Load the cleaned Target CSV
target_file = 'target_cleaned.csv'
df_target = pd.read_csv(target_file)

# Combine the two DataFrames
df_combined = pd.concat([df_wholefoods, df_target], ignore_index=True)

# Save the combined DataFrame to a new CSV
df_combined.to_csv('combined_data.csv', index=False)

print("CSV files combined successfully. Data saved to 'combined_data.csv'.")


CSV files combined successfully. Data saved to 'combined_data.csv'.


In [4]:
import pandas as pd

# Load the combined CSV file
combined_file = 'combined_data.csv'
df = pd.read_csv(combined_file)

# Function to calculate price per quantity
def calculate_price_per_quantity(row):
    try:
        # Extract numeric price and quantity
        price = float(row['Price'].replace('$', '').strip())  # Convert price to float
        quantity = row['Quantity']
        
        # Parse the numeric value and unit from quantity
        if quantity != "1":
            quantity_value, unit = quantity.split()  # Separate number and unit
            quantity_value = float(quantity_value)  # Convert quantity value to float
            
            # Calculate price per unit
            price_per_unit = price / quantity_value
            return f"${price_per_unit:.2f}/{unit}"  # Return formatted price per unit
        else:
            return row['Price']  # Return price if quantity is 1
    except:
        return "N/A"  # Return "N/A" if calculation fails

# Apply the function to create the "Price/Quantity" column
df['Price/Quantity'] = df.apply(calculate_price_per_quantity, axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv('target_and_wholefoods.csv', index=False)

print("Price/Quantity column added successfully. Data saved to 'target_and_wholefoods.csv'.")


Price/Quantity column added successfully. Data saved to 'target_and_wholefoods.csv'.


In [5]:
import pandas as pd

# Load the CSV file
combined_file = 'target_and_wholefoods.csv'  # Replace with your file path if different
df = pd.read_csv(combined_file)

# Function to calculate price per quantity
def calculate_price_per_quantity(row):
    try:
        # Extract numeric price
        price = float(row['Price'].replace('$', '').strip())
        
        # Extract numeric value and unit from Quantity
        quantity = row['Quantity']
        if quantity != "1":
            quantity_value, unit = quantity[:-2], quantity[-2:]  # Split number and unit (e.g., "2pk")
            quantity_value = float(quantity_value)  # Convert quantity value to float
            
            # Calculate price per unit
            price_per_unit = price / quantity_value
            return f"${price_per_unit:.2f}/{unit}"  # Return formatted price per unit
        else:
            return "N/A"
    except:
        return "N/A"  # Return "N/A" if calculation fails

# Apply the function to create/update the "Price/Quantity" column
df['Price/Quantity'] = df.apply(calculate_price_per_quantity, axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv('cleaned_combined_data_with_price_per_quantity.csv', index=False)

print("Price/Quantity column updated successfully. Data saved to 'cleaned_combined_data_with_price_per_quantity.csv'.")


Price/Quantity column updated successfully. Data saved to 'cleaned_combined_data_with_price_per_quantity.csv'.


In [26]:
import pandas as pd
import sqlite3

# Load the CSV file
csv_file = 'cleaned_target+wf.csv'  # Replace with your file path if different
df = pd.read_csv(csv_file)

# Connect to SQLite database (or create one if it doesn't exist)
db_file = 'products_database.db'  # Name of the database file
conn = sqlite3.connect(db_file)

# Write the DataFrame to a table in the database
table_name = 'products'
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print(f"CSV file '{csv_file}' successfully converted to SQL database '{db_file}' with table '{table_name}'.")


CSV file 'cleaned_target+wf.csv' successfully converted to SQL database 'products_database.db' with table 'products'.
